In [2]:
import pandas as pd

In [6]:
almacen = pd.read_excel('Base Almacén - ITBA.xlsx', sheet_name='lx03 04112013')
estanterias_picking = almacen[almacen['Sector'] == 'Picking']['Ubicación']

In [7]:
movimientos = pd.read_csv("Movimientos Pallet Completo.csv", encoding='ISO-8859-1', sep=';')
movimientos = movimientos[~((movimientos['Ubicproc'].isin(estanterias_picking)) | (movimientos['UbicDest'].isin(estanterias_picking)))]

C:\Users\ianda\AppData\Local\Temp\ipykernel_60988\2295009039.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  movimientos = pd.read_csv("Movimientos Pallet Completo.csv", encoding='ISO-8859-1', sep=';')


In [8]:
maestro = pd.read_excel('Mestro de Articulos - ITBA.xlsx', header=2)
mov_info = pd.merge(movimientos, maestro[['Cód. Material', 'Almacenamiento',	'Descr. Nível 1']], left_on='Material', right_on='Cód. Material', how='left')

In [27]:
mov_info["Usuario"].value_counts()

Usuario
CLHOR16    12490
CLHOR14    11179
CLHOR18    10572
CLHOR19    10513
CLHOR15    10132
CLHOR17     9860
CLHOR04     8893
CLHOR13     8766
CLHOR07     8740
CLHOR05     8537
CLHOR06     7473
CLHOR01     7215
CLHOR10     7109
CLHOR02     5873
CLHOR20     5534
CLHOR21     5498
CLHOR12     5269
CLHOR09     4985
CLHOR08     4601
CLHOR11     4145
CLHOR03     3930
CLHOR22     3852
CLHOR25     3833
CLHOR26     3193
CLHOR23     2286
CLHOR24     2140
CLOPE16     1250
CLOPE11      492
CLOPE29      250
CLOPE09      191
CLOPE33       83
CLOPE14       79
CLOPE12       72
CLOPE04       54
CLOPE31       51
CLOPE34       38
CLOPE25       21
CLOPE07       17
CLOPE22       16
CLOPE20       14
CLOPE24       12
CLOPE08       10
CLOPE18        7
CLOPE05        6
CLOPE26        4
CLOPE13        4
CLOPE06        1
Name: count, dtype: int64

In [14]:
mov_info[["Material","Mvto","Hinicio","Feconf","Almacenamiento"]].rename(columns={"Mvto":"Movimiento","Hinicio":"Hora","Feconf":"Fecha"}).sort_values(["Fecha","Hora"]).to_csv("movements.csv", index=False)